In [1]:
import os
import pandas as pd
import numpy as np
import talib as ta
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
path = "Z:\data\BTCUSDT\BTCUSDT-1h"

In [3]:
filelist = os.listdir(path)
filelist.sort()
df = pd.DataFrame()
for file in tqdm(filelist[-300:]):
    df0 = pd.read_csv(os.path.join(path, file))
    df = pd.concat([df, df0], ignore_index=True)

100%|██████████| 300/300 [00:05<00:00, 50.09it/s]


In [11]:
df1 = df[['open_time', 'open', 'high', 'low', 'close', 'volume']]
df1.columns = ['t', 'o', 'h', 'l', 'c', 'v']
df1 = df1.dropna()

In [12]:
# 计算中轨线(MB)、上轨线(UB)和下轨线(LB)
def get_cmf(df, length=14):
    df = df.copy()
    df['MFM'] = ((df['c'] - df['l']) - (df['h'] - df['c'])) / (df['h'] - df['l'])
    df['MFV'] = df['MFM'] * df['v']
    df['g1.CMF'] = df['MFV'].rolling(length).sum() / df['v'].rolling(length).sum()
    return df['g1.CMF']

In [13]:
length = 14
df1['g1.CMF'] = get_cmf(df1, length=length)
df1 = df1[length - 1:]
df1

,t,o,h,l,c,v,g1.CMF
13,1671022800000,17910.1,17964.3,17853.8,17889.9,23363.987,0.107318
14,1671026400000,17889.9,18086.8,17889.0,18055.8,55287.673,0.249787
15,1671030000000,18055.8,18111.5,18013.6,18054.8,30987.972,0.222866
16,1671033600000,18054.8,18149.6,18053.1,18096.9,37273.480,0.171189
17,1671037200000,18096.9,18146.6,18069.0,18089.3,16822.793,0.161133
...,...,...,...,...,...,...,...
7195,1696878000000,27555.0,27632.4,27526.3,27607.6,8683.886,-0.142385
7196,1696881600000,27607.7,27644.0,27542.9,27556.2,4995.344,-0.165247
7197,1696885200000,27556.1,27627.2,27542.3,27607.4,3540.588,-0.132061
7198,1696888800000,27607.4,27627.0,27562.0,27575.9,3771.446,-0.124621


In [14]:
df1.to_csv('CMFH.csv', index=False)